In [5]:
import numpy as np
import pandas as pd
import sqlite3

In [76]:
assessments = pd.read_csv('assessments.csv')
courses = pd.read_csv('courses.csv')
studentAssessment = pd.read_csv('studentAssessment.csv')
studentInfo = pd.read_csv('studentInfo.csv')
studentRegistration = pd.read_csv('studentRegistration.csv')
df_vle = pd.read_csv('vle.csv')

In [78]:
assessments.head(2)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0


In [79]:
courses.head(2)

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269


In [81]:
studentAssessment.head(2)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0


In [80]:
studentInfo.head(2)

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass


In [82]:
studentRegistration.head(2)

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN


In [83]:
df_vle.head(2)

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,NaN,NaN
1,546712,AAA,2013J,oucontent,NaN,NaN


In [3]:
select_table = 
'''
SELECT student_registration.id_student, student_info.region, student_registration.code_module, assessment.assessment_type, student_assessment.score
FROM student_registration
JOIN student_info
ON student_registration.id_student = student_info.id_student
JOIN assessment
ON student_registration.code_module = assessment.code_module
WHERE code_presentation = 2014B
'''

SyntaxError: invalid syntax (<ipython-input-3-7c8e25c89f52>, line 1)

# cleaning data frame bertujuan supaya menghilangkan eror saat di input ke dalam database

In [ ]:
assessments.head(2)

In [ ]:
assessments['date'].mean()

In [ ]:
# data kosong di kolom date di ganti dengan rata2nya karena hanya sedikit data kosong
assessments['date'].fillna(assessments['date'].mean(), inplace=True)

In [ ]:
courses.head(2)

In [ ]:
studentAssessment['score'].mean()

In [ ]:
studentAssessment.head(2)

In [ ]:
# data kosong di kolom score diganti dengan rata2nya karena tidak ada nilai/bobot yang kosong pada murid 
studentAssessment['score'].fillna(studentAssessment['score'].mean(), inplace=True)

In [ ]:
studentInfo.head(2)

In [ ]:
# data kosong di kolom unregistration diganti dengan 0 karena mungkin terdaftar
studentRegistration['date_unregistration'].fillna(0, inplace=True)

In [ ]:
# data kosong di kolom registration diganti dengan 0 karena mungkin tidak terdaftar
studentRegistration['date_registration'].fillna(0, inplace=True)

In [ ]:
studentRegistration.head(2)

In [58]:
df_vle.head(2)

,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,NaN,NaN
1,546712,AAA,2013J,oucontent,NaN,NaN


In [68]:
df_vle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6364 entries, 0 to 6363
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id_site            6364 non-null   int64  
 1   code_module        6364 non-null   object 
 2   code_presentation  6364 non-null   object 
 3   activity_type      6364 non-null   object 
 4   week_from          1121 non-null   float64
 5   week_to            1121 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 298.4+ KB


In [69]:
# hapus kolom week_from dan week_to karena jumlah yang kosong terlalu banyak 
hapus = ['week_from', 'week_to']
df_vle.drop(hapus, inplace=True, axis=1 )

In [70]:
df_vle.head()

,id_site,code_module,code_presentation,activity_type
0,546943,AAA,2013J,resource
1,546712,AAA,2013J,oucontent
2,546998,AAA,2013J,resource
3,546888,AAA,2013J,url
4,547035,AAA,2013J,resource


# create table dan insert table dari data frame yg telah di buat di atas

In [7]:
# fungsi untuk connect ke database
def create_connection(path):
    connection=sqlite3.connect(path)
    return(connection)

connection = create_connection('latihan.DB')

In [8]:
# query untuk non select
def execute_command(conn,query):    
    cur = conn.cursor()    
    cur.execute(query)    
    conn.commit()

In [9]:
# query untuk select
def select_query(conn,query):
    cur = conn.cursor()
    cur.executed(query)
    rows = cur.fetchall()
    
    for row in rows:
        print(row)

# pandas.read_sql_query(query,connection)
# Q = select * from teachers;
# pd.read_sql_query(Q,connection)

# 1. membuat dan isi tabel assessments

In [ ]:
delete='''DROP TABLE assessments'''
execute_command(connection,delete)

In [ ]:
table_ass ='''
CREATE TABLE assessments(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code_module varchar (50),
    code_presentation varchar(50),
    id_assessment numeric,
    assessment_type varchar(50),
    date numeric,
    weight numeric
);
'''

execute_command(connection, table_ass)

In [ ]:
for i in range (len(assessments)):
    ass=assessments.loc[i]
    create_row='''
    INSERT INTO assessments('code_module', 'code_presentation', 'id_assessment', 'assessment_type', 'date', 'weight')
    VALUES('{}','{}',{},'{}',{},{})
    '''.format(ass['code_module'],ass['code_presentation'],ass['id_assessment'],ass['assessment_type'],ass['date'],ass['weight'])
    execute_command(connection,create_row)

In [ ]:
pd.read_sql_query('select * from assessments',connection).head(5)

In [ ]:
assessments.head(5)

# 2. membuat dan isi tabel courses

In [ ]:
delete='''DROP TABLE courses'''
execute_command(connection,delete)

In [ ]:
table_cour='''
CREATE TABLE courses(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code_module varchar(50),
    code_presentation varchar(50),
    module_presentation_length numeric
);
'''
execute_command(connection,table_cour)

In [ ]:
for i in range (len(courses)):
    cour=courses.loc[i]
    create_row='''
    INSERT INTO courses('code_module', 'code_presentation', 'module_presentation_length')
    VALUES('{}','{}',{})'''.format(cour['code_module'], cour['code_presentation'], cour['module_presentation_length'])
    
    execute_command(connection,create_row)

# 3. membuat dan isi tabel studentAssessment

In [ ]:
delete='''DROP TABLE student_assessment'''
execute_command(connection,delete)

In [ ]:
table_student_ass='''
CREATE TABLE student_assessment(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_assessment numeric,
    id_student numeric,
    date_submitted numeric,
    is_banked numeric,
    score numeric
);
'''
execute_command(connection,table_student_ass)

In [ ]:
for i in range (len(studentAssessment)):
    student_ass=studentAssessment.loc[i]
    create_row='''
    INSERT INTO student_assessment('id_assessment', 'id_student', 'date_submitted', 'is_banked', 'score')
    VALUES({},{},{},{},{})'''.format(student_ass['id_assessment'], student_ass['id_student'], 
                                     student_ass['date_submitted'], student_ass['is_banked'], student_ass['score'])
    
    execute_command(connection,create_row)

# 4. membuat dan isi tabel studentInfo

In [ ]:
delete='''drop table student_info'''
execute_command(connection, delete)

In [ ]:
table_student_inf='''
CREATE TABLE student_info(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code_module varchar(50),
    code_presentation varchar(50),
    id_student numeric,
    gender varchar(50),
    region varchar(50),
    highest_education varchar(50),
    imd_band varchar (50),
    age_band varchar(50),
    num_of_prev_attempts numeric,
    studied_credits numeric,
    disability varchar(50),
    final_result varchar(50)    
);
'''
execute_command(connection,table_student_inf)

In [ ]:
for i in range(len(studentInfo)):
    student_info=studentInfo.loc[i]
    create_row='''
    INSERT INTO student_info('code_module', 'code_presentation', 'id_student', 'gender', 'region', 'highest_education',
    'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability', 'final_result')
    VALUES ('{}','{}',{},'{}','{}','{}','{}','{}',{},{},'{}','{}')
    '''.format(student_info['code_module'], student_info['code_presentation'], student_info['id_student'], student_info['gender'],
        student_info['region'], student_info['highest_education'], student_info['imd_band'], student_info['age_band'], student_info['num_of_prev_attempts'],
        student_info['studied_credits'], student_info['disability'], student_info['final_result'])
    execute_command(connection,create_row)

# 5. membuat dan isi tabel studentRegistration

In [ ]:
delete='''DROP TABLE student_registration'''
execute_command(connection,delete)

In [4]:
table_student_regis='''
CREATE TABLE student_registration(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    code_module varchar(50), 
    code_presentation varchar(50),
    id_student numeric, 
    date_registration numeric ,
    date_unregistration numeric);
    '''
execute_command(connection,table_student_regis)

NameError: name 'execute_command' is not defined

In [ ]:
for i in range(len(studentRegistration)):
    student_regis=studentRegistration.loc[i]
    create_row='''
    INSERT INTO student_registration('code_module', 'code_presentation', 'id_student', 'date_registration', 'date_unregistration')
    VALUES('{}','{}',{},{},{})
    '''.format(student_regis['code_module'], student_regis['code_presentation'], student_regis['id_student'], student_regis['date_registration'],
              student_regis['date_unregistration'],)
    execute_command(connection,create_row)

# 6. membuat dan isi tabel vle

In [71]:
delete='''DROP TABLE vle'''
execute_command(connection,delete)

In [72]:
table_vle='''
CREATE TABLE vle(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    id_site numeric,
    code_module varchar(50),
    code_presentation varchar(50),
    activity_type varchar(50),
    week_from varchar(50),
    week_to varchar(50)
    );'''

execute_command(connection,table_vle)

In [74]:
for i in range(len(df_vle)):
    vle=df_vle.loc[i]
    create_row='''
    INSERT INTO vle('id_site', 'code_module', 'code_presentation', 'activity_type')
    VALUES ({},'{}','{}','{}')
    '''.format(vle['id_site'], vle['code_module'], vle['code_presentation'], vle['activity_type'])
    execute_command(connection,create_row)